# Scripts for mapping fastq data, variant calling, imputation of missing data and haplotype phasing

## Before you begin

#### Copy the tools to 'bin/' and in each cell below, define paths to the tools. Adjust '$ncpu', the number of computing threads used, within the script if necessary.

### Prepare the reference genome for mapping

In [ ]:
cd ../.. #star_protocols_saimaa
work=$(pwd)
apps=$work/bin

bwa=$apps/bwa
samtools=$apps/samtools

refd=$work/data/reference

refs=$refd/norppa_12122017

gunzip $refs.fa.gz

$samtools faidx $refs.fa
$samtools dict $refs.fa > $refs.dict

$bwa index $refs.fa


## 1.Mapping using BWA

### The following script is in 'scripts/ map_fastq.sh'.

In [ ]:
cd ../.. #star_protocols_saimaa
work=$(pwd)
apps=$work/bin

bwa=$apps/bwa
samtools=$apps/samtools
gatk3=$apps/gatk3

ncpu=4

sample=$1
fastq1=$2
fastq2=$3

temp=$work/processed_data/1.mapping-variantcalling/temp
bams=$work/processed_data/1.mapping-variantcalling/bams

mkdir -p $temp
mkdir -p $bams 

log=$work/bams/$sample-`date +%F-%T`.log

RG="@RG\tID:${sample}\tSM:${sample}\tPL:ILLUMINA\tLB:LIB"

( $bwa mem -M -R $RG -t $ncpu $refs.fa $fastq1 $fastq2 \
  | $samtools view -h -b -o $temp/${sample}_align.bam - ) 2>> $log

$samtools fixmate -@ $ncpu -m $temp/${sample}_align.bam $temp/${sample}_fixm.bam &>> $log

$samtools sort -@ $ncpu $temp/${sample}_fixm.bam -o $temp/${sample}_sort.bam &>> $log

$samtools index $temp/${sample}_sort.bam &>> $log

$gatk3 -T RealignerTargetCreator \
  -R $refs.fa -I $temp/${sample}_sort.bam -o $temp/${sample}.intervals  \
  -nt $ncpu &>> $log
  
$gatk3 -T IndelRealigner \
  -R $refs.fa -I $temp/${sample}_sort.bam -o $temp/${sample}_realign.bam \
  -targetIntervals $temp/${sample}.intervals &>> $log

$samtools markdup -@ $ncpu $temp/${sample}_realign.bam $bams/${sample}_markdup.bam
$samtools index $bams/${sample}_markdup.bam

rm $temp/${sample}*


## 2.Genotype calling

### 2.1. ANGSD

In [ ]:
cd ../.. #star_protocols_saimaa
work=$(pwd)
apps=$work/bin

angsd=$apps/angsd

ncpu=4

output=$work/processed_data/1.mapping-variantcalling/angsd/saimaa

bams=$work/processed_data/1.mapping-variantcalling/bams
bamsfile=$work/processed_data/1.mapping-variantcalling/angsd/bams.txt
ls $bams/*bam | sort -V > $bamsfile

#Running ANGSD

$angsd -GL 2 -doGlf 2 -doDepth 1 -doCounts 1 -doMajorMinor 1 -doMaf 2 \
  -minMaf 0.05 -SNP_pval 1e-6 -uniqueOnly 1 -minMapQ 30 -minQ 20 -skipTriallelic 1 \
  -nThreads $ncpu -minInd 20 \
  -bam $bamsfile -out $output &> $output.log


### 2.2. PCAngsd

In [ ]:
cd ../.. #star_protocols_saimaa
work=$(pwd)
apps=$work/bin

pangsd="python3 $apps/pcangsd-v.0.99/pcangsd.py"

file=$work/processed_data/1.mapping-variantcalling/angsd/saimaa

$pcangsd -beagle $file.beagle.gz -e 4 -o $file -post_save

awk -f $work/scripts/1.mapping-variantcalling/beagle2vcf.awk $file.post.beagle | bgzip -c > $file.tmp.vcf.gz

$bcftools reheader -s $work/data/saimaa_IDs.txt $file.tmp.vcf.gz -o $file.vcf.gz


### 2.3. Beagle: imputation

In [ ]:
cd ../.. #star_protocols_saimaa
work=$(pwd)
apps=$work/bin

beagle="java -Xmx40G -jar $apps/beagle.08Jun17.d8b.jar"

input=$work/processed_data/1.mapping-variantcalling/angsd/saimaa.vcf.gz

imputed=$work/processed_data/1.mapping-variantcalling/beagle/saimaa_imputed
phased=$work/processed_data/1.mapping-variantcalling/beagle/saimaa_phased

$beagle nthreads=10 gl=$input out=$imputed &> $imputed.err

$beagle nthreads=10 gt=$imputed.vcf.gz out=$phased &> $phased.err
